# Capstone Project Get Data

Load the data CSV files into Pandas data frames and import modules that are commonly in need

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

lego = pd.read_csv('assets/datasets/legosets.csv')
set_pieces = pd.read_csv('assets/datasets/set_pieces.csv')
pieces = pd.read_csv('assets/datasets/pieces.csv')

print lego.info()
print ''
print '---------------------------'
print ''
print set_pieces.info()
print ''
print '---------------------------'
print ''
print pieces.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6172 entries, 0 to 6171
Data columns (total 14 columns):
Item_Number     6172 non-null object
Name            6172 non-null object
Year            6172 non-null int64
Theme           6172 non-null object
Subtheme        3966 non-null object
Pieces          6060 non-null float64
Minifigures     3500 non-null float64
Image_URL       6172 non-null object
GBP_MSRP        4192 non-null float64
USD_MSRP        5817 non-null float64
CAD_MSRP        1982 non-null float64
EUR_MSRP        1773 non-null float64
Packaging       6172 non-null object
Availability    6172 non-null object
dtypes: float64(6), int64(1), object(7)
memory usage: 675.1+ KB
None

---------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547156 entries, 0 to 547155
Data columns (total 5 columns):
set_id      547156 non-null object
piece_id    547156 non-null object
num         547156 non-null int64
color       547156 non-null int64
type        547156 non-

From the above information we can see that set_pieces and pieces have no missing values. Therefore they will be stored as a PostgreSQL table in a database. The PostgreSQL database will be created in the localshost. Data frames will be stored as tables inside the database with the same name.

In [4]:
from sqlalchemy import create_engine
local_engine = create_engine('postgresql://localhost:5432')
conn = local_engine.connect()
conn.execute("commit")
conn.execute("CREATE DATABASE capstone")
conn.close()
engine_capstone=create_engine('postgresql://localhost/capstone')
set_pieces.to_sql('set_pieces',engine_capstone,index=False)
pieces.to_sql('pieces',engine_capstone,index=False)

It is apparent that there are quite a lot of missing values in the lego data frame. Therefore the first approach is to try filling in the missing values by accessing the API provided by Brickset.com. In the same time some more useful information will be recorded. These information include.
- Number of members owing this item
- Number of members wanting this item
- Rating of the item in Brickset.com
- Theme Group of item
- Number of ratings in Brickset.com for the item

Also it is possible to see that there are a lot of missing values in the MSRP columns. Therefore it would be good it check if any of the can be filled.

Upon the examinations of the lego data, it is discoverd that some item numbers are repeated. For example Lego has a series of products called Collectable Minifigures. These minifigures are selaed in foil packs and sold seperately. Customers cannot see through the foil packs and this adds an element of excitment similar to lucky draws. However this means the different minifigures have the same item numbers which is indistinguishable. Fortunately it is discovered that in the Image_URL column, the exact Brickset item number can be found. Regular expressions are used to extract such information which is then stored in a new column called 'query_id'.

In [9]:
for i in range(len(lego)):
    try:
        lego.ix[i,'query_id'] = re.search(r'images/(.+-\d+)',lego.ix[i,'Image_URL']).group(1)
    except: 
        print 'no'

5 new columns are created before the data is gathered. This is because it is considered to be a more effective way to achieve data gathering. Normally one would use the .apply() method on the 'query_id' column to extract information and directly assign to a new variable/column. However since this is only viable for 1 column each time and there are 5 pieces of information that needs to be obtained, it is decided that it would be easier to modify existing columns. This approach reduces the number of get requests performed which speeds up the process and reduce the loading on the server. The considerations here being that it is not the most ethical to hit a server too frequently in a short period of time and by doing so might result in an I.P address ban which is undesirable. 

In [10]:
lego['wanted'] = 0
lego['owned'] = 0
lego['rating'] = 0
lego['theme_grp'] = 0
lego['review_num'] = 0
lego.head()

,Item_Number,Name,Year,Theme,Subtheme,Pieces,Minifigures,Image_URL,GBP_MSRP,USD_MSRP,CAD_MSRP,EUR_MSRP,Packaging,Availability,wanted,owned,rating,theme_grp,review_num,query_id
0,10246,Detective's Office,2015,Advanced Models,Modular Buildings,2262.0,6.0,http://images.brickset.com/sets/images/10246-1...,132.99,159.99,199.99,149.99,Box,Retail - limited,0,0,0,0,0,10246-1
1,10247,Ferris Wheel,2015,Advanced Models,Fairground,2464.0,10.0,http://images.brickset.com/sets/images/10247-1...,149.99,199.99,229.99,179.99,Box,Retail - limited,0,0,0,0,0,10247-1
2,10248,Ferrari F40,2015,Advanced Models,Vehicles,1158.0,NaN,http://images.brickset.com/sets/images/10248-1...,69.99,99.99,119.99,89.99,Box,LEGO exclusive,0,0,0,0,0,10248-1
3,10249,Toy Shop,2015,Advanced Models,Winter Village,898.0,NaN,http://images.brickset.com/sets/images/10249-1...,59.99,79.99,NaN,69.99,Box,LEGO exclusive,0,0,0,0,0,10249-1
4,10581,Ducks,2015,Duplo,Forest Animals,13.0,1.0,http://images.brickset.com/sets/images/10581-1...,9.99,9.99,12.99,9.99,Box,Retail,0,0,0,0,0,10581-1


The cell below shows the simple get_info function to obtain the information. The method is rather simple by getting the information of certain Lego sets using the Brickset API. Then various functions are applied to extract the desired information by apply regular expression searches.

In [7]:
import requests
def get_info(x):
    print x
    URL = 'http://brickset.com/api/v2.asmx/getSets?apiKey=yn4G-wvgQ-wNct&userHash=&query=&theme=&subtheme=&setNumber='+x+'&year=&owned=&wanted=&orderBy=&pageSize=&pageNumber=&userName='
    response = requests.get(URL)
    res = response.content
    get_wanted(x,res)
    get_owned(x,res)
    get_theme_grp(x,res)
    get_rating(x,res)
    get_review_num(x,res)
    get_usd(x,res)
    get_gbp(x,res)
    get_cad(x,res)
    get_eur(x,res)

In [8]:
def get_wanted(q,res):
    try:
        x = re.search(r'<wantedByTotal>(.+)</wantedByTotal>',res).group(1)
    except:
        x = '1'
    lego.ix[lego['query_id'] == q, 'wanted'] = x
def get_owned(q,res):
    try:
        x = re.search(r'<ownedByTotal>(.+)</ownedByTotal>',res).group(1)
    except:
        x = '1'
    lego.ix[lego['query_id'] == q, 'owned'] = x
def get_theme_grp(q,res):
    try:
        x = re.search(r'<themeGroup>(.+)</themeGroup>',res).group(1)
    except:
        x = 'not_applicable'
    lego.ix[lego['query_id'] == q, 'theme_grp'] = x
def get_rating(q,res):
    try:
        x = re.search(r'<rating>(.+)</rating>',res).group(1)
    except:
        x = '0'
    lego.ix[lego['query_id'] == q, 'rating'] = x
def get_review_num(q,res):
    try:
        x = re.search(r'<reviewCount>(.+)</reviewCount>',res).group(1)
    except:
        x = '0'
    lego.ix[lego['query_id'] == q, 'review_num'] = x
def get_usd(q,res):
    try:
        x = re.search(r'<USRetailPrice>(.+)</USRetailPrice>',res).group(1)
    except:
        x = np.nan
    lego.ix[lego['query_id'] == q, 'USD_MSRP'] = x
def get_gbp(q,res):
    try:
        x = re.search(r'<UKRetailPrice>(.+)</UKRetailPrice>',res).group(1)
    except:
        x = np.nan
    lego.ix[lego['query_id'] == q, 'GBP_MSRP'] = x
def get_cad(q,res):
    try:
        x = re.search(r'<CARetailPrice>(.+)</CARetailPrice>',res).group(1)
    except:
        x = np.nan
    lego.ix[lego['query_id'] == q, 'CAD_MSRP'] = x
def get_eur(q,res):
    try:
        x = re.search(r'<EURetailPrice>(.+)</EURetailPrice>',res).group(1)
    except:
        x = np.nan
    lego.ix[lego['query_id'] == q, 'EUR_MSRP'] = x

Running the get_info function

In [11]:
lego.query_id.apply(get_info)

10246-1
10247-1
10248-1
10249-1
10581-1
10582-1
10583-1
10584-1
10585-1
10586-1
10587-1
10589-1
10590-1
10591-1
10592-1
10593-1
10594-1
10595-1
10596-1
10597-1
10599-1
10600-1
10601-1
10602-1
10603-1
10604-1
10605-1
10606-1
10607-1
10608-1
10615-1
10616-1
10617-1
10618-1
10622-1
10677-1
10679-1
10680-1
10683-1
10684-1
10685-1
10686-1
10687-1
10692-1
10693-1
10694-1
10695-1
10696-1
10697-1
10698-1
10699-1
10700-1
10701-1
21022-1
21023-1
21024-1
21119-1
21120-1
21121-1
21122-1
21301-1
21302-1
21303-1
30202-1
30204-1
30256-1
30272-1
30274-1
30275-1
30283-1
30285-1
30291-1
30292-1
30303-1
30311-1
30313-1
30314-1
30315-1
31027-1
31028-1
31029-1
31030-1
31031-1
31032-1
31033-1
31034-1
31035-1
31036-1
31037-1
31038-1
31039-1
40120-1
40121-1
40122-1
40140-1
40153-1
40154-1
40155-1
40156-1
40158-1
41060-1
41061-1
41062-1
41063-1
41071-1
41072-1
41073-1
41074-1
41075-1
41076-1
41077-1
41078-1
41085-1
41086-1
41087-1
41088-1
41089-1
41090-1
41091-1
41092-1
41093-1
41094-1
41095-1
41097-1
41098-1


0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
6142    None
6143    None
6144    None
6145    None
6146    None
6147    None
6148    None
6149    None
6150    None
6151    None
6152    None
6153    None
6154    None
6155    None
6156    None
6157    None
6158    None
6159    None
6160    None
6161    None
6162    None
6163    None
6164    None
6165    None
6166    None
6167    None
6168    None
6169    None
6170    None
6171    None
Name: query_id, dtype: object

Checking the results.

In [16]:
lego.head()

,Item_Number,Name,Year,Theme,Subtheme,Pieces,Minifigures,Image_URL,GBP_MSRP,USD_MSRP,CAD_MSRP,EUR_MSRP,Packaging,Availability,wanted,owned,rating,theme_grp,review_num,query_id
0,10246,Detective's Office,2015,Advanced Models,Modular Buildings,2262.0,6.0,http://images.brickset.com/sets/images/10246-1...,132.99,159.99,199.99,149.99,Box,Retail - limited,5008,10155,4.75,Model making,4,10246-1
1,10247,Ferris Wheel,2015,Advanced Models,Fairground,2464.0,10.0,http://images.brickset.com/sets/images/10247-1...,149.99,199.99,229.99,179.99,Box,Retail - limited,3219,4807,5,Model making,3,10247-1
2,10248,Ferrari F40,2015,Advanced Models,Vehicles,1158.0,NaN,http://images.brickset.com/sets/images/10248-1...,69.99,99.99,119.99,89.99,Box,LEGO exclusive,2030,3943,5,Model making,4,10248-1
3,10249,Toy Shop,2015,Advanced Models,Winter Village,898.0,NaN,http://images.brickset.com/sets/images/10249-1...,59.99,79.99,99.99,69.99,Box,LEGO exclusive,2046,4794,4,Model making,1,10249-1
4,10581,Ducks,2015,Duplo,Forest Animals,13.0,1.0,http://images.brickset.com/sets/images/10581-1...,9.99,9.99,12.99,9.99,Box,Retail,117,444,0,Pre-school,0,10581-1


Saving the result CSV to a PostgreSQL table. It is because one can always load the table from the database to perform analysis instead of needing to download the extra information again.

In [14]:
lego.to_sql('lego',engine_capstone,index=False)

In [15]:
lego.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6172 entries, 0 to 6171
Data columns (total 20 columns):
Item_Number     6172 non-null object
Name            6172 non-null object
Year            6172 non-null int64
Theme           6172 non-null object
Subtheme        3966 non-null object
Pieces          6060 non-null float64
Minifigures     3500 non-null float64
Image_URL       6172 non-null object
GBP_MSRP        4215 non-null object
USD_MSRP        5800 non-null object
CAD_MSRP        1987 non-null object
EUR_MSRP        1800 non-null object
Packaging       6172 non-null object
Availability    6172 non-null object
wanted          6172 non-null object
owned           6172 non-null object
rating          6172 non-null object
theme_grp       6172 non-null object
review_num      6172 non-null object
query_id        6172 non-null object
dtypes: float64(2), int64(1), object(17)
memory usage: 964.4+ KB


The about info summary shows that there are still missing values in certain columns. However we now have the necessary columns to perform the analysis which will be carried out in another Notebook.